<a href="https://colab.research.google.com/github/OdysseusPolymetis/chr2023/blob/main/4_bert_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
def encode_sequence(sequence, tokenizer, model):
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [8]:
sequence_1 = "I have a pen"
sequence_2 = "This is a pen"
sequence_3 = "I have an apple"

In [9]:
vec_1 = encode_sequence(sequence_1, tokenizer, model)
vec_2 = encode_sequence(sequence_2, tokenizer, model)
vec_3 = encode_sequence(sequence_3, tokenizer, model)

In [10]:
similarity_1_2 = cosine_similarity(vec_1.detach().numpy(), vec_2.detach().numpy())[0][0]
similarity_1_3 = cosine_similarity(vec_1.detach().numpy(), vec_3.detach().numpy())[0][0]


similarity_1_2, similarity_1_3

(0.7957163, 0.87849987)

In [11]:
import requests

def download_text(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [12]:
url_shakespeare = "https://www.gutenberg.org/files/1112/1112-0.txt"
url_shaw = "https://www.gutenberg.org/files/2304/2304.txt"

In [13]:
text_shakespeare = download_text(url_shakespeare)
text_shaw = download_text(url_shaw)

In [14]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
sentences_shakespeare = sent_tokenize(text_shakespeare)
sentences_shaw = sent_tokenize(text_shaw)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
def encode_sentences_in_batches(sentences, tokenizer, model, batch_size=32):
    all_embeddings = []

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings)

    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

In [ ]:
batch_size = 32
encoded_shakespeare = encode_sentences_in_batches(sentences_shakespeare, tokenizer, model, batch_size)
encoded_shaw = encode_sentences_in_batches(sentences_shaw, tokenizer, model, batch_size)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(encoded_set1, encoded_set2):
    similarities = cosine_similarity(encoded_set1, encoded_set2)
    return similarities


similarities = calculate_similarity(encoded_shakespeare, encoded_shaw)

most_similar_pairs = np.unravel_index(np.argsort(similarities, axis=None)[::-1], similarities.shape)

for i in range(10):
    shakespeare_idx, shaw_idx = most_similar_pairs[0][i], most_similar_pairs[1][i]
    print(f"Shakespeare phrase: {sentences_shakespeare[shakespeare_idx]}")
    print(f"Shaw phrase: {sentences_shaw[shaw_idx]}")
    print(f"Similarity: {similarities[shakespeare_idx][shaw_idx]}\n")

